In [1]:
from transit_parser import *

## NJ Transit 

In [2]:
train_name = './test/' + '2018_01_29_3891'

In [3]:
t = TrainParser(train_name)
times = t.get_stop_times()
rows = t.get_rows(times)
df = t.get_df(rows)
performance = t.join_schedule(df)

In [4]:
performance[['from', 'to','status', 'time', 'expected','train_id']]

,from,to,status,time,expected,train_id
stop_num,,,,,,
1,New York Penn Station,New York Penn Station,Departed,2018-01-29 22:10:04,2018-01-29 22:11:00,3891
2,New York Penn Station,Secaucus Upper Lvl,Departed,2018-01-29 22:20:11,2018-01-29 22:20:00,3891
3,Secaucus Upper Lvl,Newark Penn Station,Departed,2018-01-29 22:31:09,2018-01-29 22:30:00,3891
4,Newark Penn Station,Newark Airport,Departed,2018-01-29 22:36:10,2018-01-29 22:36:00,3891
5,Newark Airport,Rahway,Departed,2018-01-29 22:46:10,2018-01-29 22:46:00,3891
6,Rahway,Metropark,Departed,2018-01-29 22:53:07,2018-01-29 22:52:00,3891
7,Metropark,Metuchen,Departed,2018-01-29 22:57:10,2018-01-29 22:56:00,3891
8,Metuchen,Edison,Departed,2018-01-29 23:01:05,2018-01-29 23:01:00,3891
9,Edison,New Brunswick,Departed,2018-01-29 23:05:06,2018-01-29 23:05:00,3891


## Amtrak

In [5]:
train_id = './test/' + '2018_01_29_A177'

In [6]:
a_t = TrainParser(train_id)
a_times = a_t.get_stop_times()
a_rows = a_t.get_rows(a_times)
a_df = a_t.get_df(a_rows)
a_performance = a_t.join_schedule(a_df)

In [8]:
a_performance[['from', 'to','status', 'time', 'expected','train_id']]

,from,to,status,time,expected,train_id
stop_num,,,,,,
1,New York Penn Station,New York Penn Station,Departed,2018-01-29 22:05:51,None,A177
2,New York Penn Station,Newark Penn Station,Departed,2018-01-29 22:22:01,None,A177
3,Newark Penn Station,Metropark,Departed,2018-01-29 22:38:08,None,A177
4,Metropark,Trenton,Departed,2018-01-29 23:01:02,None,A177
5,Trenton,Philadelphia,Departed,2018-01-29 23:30:07,None,A177
